In [1]:
import numpy as np

# Code for getting the restricted coboundaryMatrix
# The code assumes that the boundary matrix is just called D, replace D if you're calling it something else
restrictedD = D[nv:nv + ne, nv + ne:len(D)]
coboundaryMatrix = np.flip(restrictedD).T
# For a matrix A, you can get its antitranspose using the command np.flip(A).T

NameError: name 'D' is not defined

In [11]:
# Reduction Algorithm
# Inputs: Matrix R (make sure to set it equal to D or D antitranspose)
# Outputs: Prints the barcode (optional, uncomment code) and returns the new R and (optional, uncomment return code) V

def reductionAlgorithm(R):
    numRows = len(R)
    V = np.eye(numRows)
    pivotRows, pivotEntries = [[] for i in range (numRows)], [[] for i in range (numRows)]

    for i in range(numRows):
        h = 0
        while (h == 0):
            pivotRow = np.inf
            for j in range(numRows - 1, -1, -1):
                if R[j][i] != 0:
                    pivotRow = j
                    break;
            if pivotRow == np.inf:
                pivotEntries[i] = 0
            else:
                pivotEntries[i] = R[pivotRow][i]
            pivotRows[i] = pivotRow
            if pivotRow == np.inf or all(pivotRows[k] != pivotRows[i] for k in range(i)):
                h = 1
                break;
            for k in range(i):
                if pivotRows[k] == pivotRows[i]:
                    c = pivotEntries[i] // pivotEntries[k]
                    for l in range(numRows):
                        R[l][i] -= c * R[l][k]
                        V[l][i] -= c * V[l][k]
                        
    '''for i in range (numRows):
        if any(pivotRows[j] == i or pivotRows[i] == j for j in range(numRows)):
            for k in range(numRows):
                if (pivotRows[k] == i):
                    print(f"[{i + 1}, {k + 1})")
                    break;
        else:
            print(f"[{i + 1}, inf)")'''
            
    return R#, V

In [1]:
# Row Detection Algorithm
# Inputs: n x m matrix A, n x 1 vector b
# Outputs: Row in which matrix no longer has a solution (going bottom up)
# Call: rowDetect(A, b) (checkSolution is only defined to be used inside of that function)

def checkSolution(A, b):
    # Calculate the rank of the augmented matrix [A | b]
    augmented_matrix = np.column_stack((A, b))
    rank_A = np.linalg.matrix_rank(A)
    rank_augmented = np.linalg.matrix_rank(augmented_matrix)
    num_columns = A.shape[1]
    num_rows = A.shape[0]

    if rank_A == rank_augmented:
        if rank_A == num_columns:
            return 1
        elif rank_A < num_columns:
            return 1
    else:
        if rank_A < rank_augmented:
            return 0
    
    return "Unable to determine the solution status."

def rowDetect(A, b):
    iMax = len(A) + 1
    iMin = 1
    i = len(A) // 2
    running = True
    while(running):
        sol = checkSolution(A[-i:], b[-i:])
        if sol == 0:
            if checkSolution(A[-(i - 1):], b[-(i - 1):]) == 1:
                row = len(A) - i
                running = False
                break;
            else:
                if i < iMax:
                    iMax = i
                    i = (iMax + iMin) // 2
        if sol == 1:
            if i > iMin:
                iMin = i
                i = (iMax + iMin) // 2
    return row

In [6]:
# Boundary Matrix Algorithm
# Inputs: Ordered list of simplices (should be ordered based on the Rips filtration, in this case)
# Outputs: Boundary matrix D
# Call: boundaryMatrix(simplices) (boundary is only defined to be used inside of that function)

def boundary(simplex):
    boundaries = []
    for i in range(len(simplex)):
        boundaries.append(simplex[:i] + simplex[i+1:])
    return boundaries

def boundaryMatrix(simplices):
    D = []
    for i in simplices:
        subD = []
        boundD = boundary(i)
        for j in simplices:
            for k in range(len(boundD)):
                if boundD[k] == j:
                    if k % 2 == 0:
                        subD.append(1)
                    else:
                        subD.append(-1)
                    break;   
            else:
                subD.append(0)
        D.append(subD)
    D = np.array(D)
    D = D.T
    return D

In [8]:
# Cup Product Algorithm
# Inputs: Cocycles phi and psi (restricted so they have the same # of rows) and list of triangles at that value alpha
# Outputs: Cup product vector (feel free to edit the code if you want the triangles to be included as well)

def cupProduct(phi, psi, triangleList):
    cupProduct = []
    for i in range(len(triangleList)):
        phiVal = 0
        psiVal = 0
        for j in range(len(phi)):
            if phi[j][0] == triangleList[i][0] and phi[j][1] == triangleList[i][1]:
                phiVal = phi[j][2]
                break;
        for k in range(len(psi)):
            if psi[k][0] == triangleList[i][1] and psi[k][1] == triangleList[i][2]:
                psiVal = psi[k][2]
                break;
        cupProduct.append(phiVal * psiVal)
    cupProduct = np.array(cupProduct).T
    return cupProduct